AGUDELO Julian et PONCE Afonso - 2023
# TP - Site web de rencontre de proteins

A continuation, nous allons nous consacrer à la conception d'un site web fondé sur l'ontologie GO, une ontologie très utilisée en bio-informatique.

### Sur Go :
L'ontologie GO se compose de trois parties: les processus biologiques, les fonctions moléculaires, et les composants cellulaires.
La troisième composante se concentre sur la description des éléments constitutifs d'une cellule, tels que les membranes, le noyau, et les organites.
La gestion de cette troisième partie s'avère complexe en raison de la coexistence de deux types de hiérarchies. D'une part, une hiérarchie d'héritage classique avec des relations "est-un" caractérise la structure. D'autre part, une hiérarchie basée sur la relation "partie-de", connue sous le nom de méronymie, vise à décomposer la cellule en sous-parties, puis en sous-sous-parties, avec la cellule entière en tant que racine et les parties indivisibles en tant que feuilles.

GO utilise des identifiants arbitraires qui ne sont pas directement compréhensibles par l’humain. Le tableau suivant récapitule les identifiants GO dont nous aurons besoin par la suite :

| Identifiant GO    | Label                 | Description                |
| --------          | -------               | -------                    |
| GO_0005575        | cellular_component    | Composant de la cellule    |
| BFO_0000050       | part_of               | Partie de                  |
| BFO_0000051       | has_part              | A pour partie              |

### Sur les fonctionnalités attendues du site web :
Ce site offrira la possibilité d'entrer deux noms de protéines et de déterminer les compartiments cellulaires dans lesquels elles pourraient éventuellement interagir. D'un point de vue biologique, cette fonctionnalité revêt une importance particulière, car des protéines dépourvues d'un "site de rencontre" commun ne peuvent pas interagir entre elles.


## 1. Explorant GO et gerent des relations á travers de nouvelles méthodes

In [ ]:
# Commençons par importer les bibliothèques nécessaires
from flask import Flask
from flask import request
from owlready2 import *
import mygene
import werkzeug.serving

Owlready2 stocke tous les triples dans un objet 'World', et peut gérer plusieurs mondes en parallèle. 'default_world' est le monde utilisé par défaut.

Owlready2 utilise un quadstore optimisé (comme les [Triplestore ou RDF store](https://en.wikipedia.org/wiki/Triplestore) , mais avec 4 composants) . Par défaut, le quadstore est stocké en mémoire, mais il peut également être stocké dans un fichier SQLite3. Cela permet la persistance des données.

In [ ]:
# La méthode .set_backend() de World définit le nom de fichier SQLite3 associé au quadstore, par exemple :
default_world.set_backend(filename = "quadstore.sqlite3")

# Une nouvelle ontologie vide peut être obtenue avec la fonction get_ontology(),
# elle prend un seul paramètre, l'IRI de l'ontologie. L'IRI est une sorte d'URL,
# les IRI sont utilisés comme identifiants pour les ontologies.
go = get_ontology("http://purl.obolibrary.org/obo/go.owl#").load()

# Les ontologies peuvent définir des entités situées dans d'autres espaces de noms. 
# L'ontologie GO en est un exemple : l'IRI de l'ontologie est "http://purl.obolibrary.org/obo/go.owl", 
# mais les IRI des entités GO ne sont pas de la forme "http://purl.obolibrary.org/obo/go.owl#GO_entity" 
# mais "http://purl.obolibrary.org/obo/GO_entity" (notez l'absence de "go.owl#").
# These entities can be accessed in Owlready2 using a "namespace". 
# The get_namepace(base_iri) global function returns a "namespace" for the given base IRI.
obo = go.get_namespace("http://purl.obolibrary.org/obo/")
default_world.save()

Montrons les entités GO, pour voir comment Owlready2 les rend. 

In [ ]:
print(obo.GO_0005575)
print(obo.BFO_0000050)
print(obo.BFO_0000051)

Veuillez noter que les informations que nous obtenons ne sont pas très descriptives.

La fonction globale 'set_render_func()' peut être utilisée pour spécifier comment Owlready2 rend les entités, c'est-à-dire comment elles sont converties en texte lors de leur impression. 'set_render_func()' accepte un seul paramètre, une fonction qui prend une entité et retourne une chaîne de caractères.

<span style="color:red">Connaissant la structure du GO, proposer une fonction de rendu qui affiche le nom de l'entité et son libellé.
</span>

In [ ]:
def mon_rendu(entity):
    # ICI VOTRE CODE
set_render_func(mon_rendu)

Testez votre fonction avec le morceau de code ci-dessous.

In [ ]:
print(obo.GO_0005575)
print(obo.BFO_0000050)
print(obo.BFO_0000051)
print(list(go.classes())[:3])

'''
Réponses attendues:

GO_0005575:cellular_component
BFO_0000050:part of
BFO_0000051:has part 
[GO_0003674:molecular_function, GO_0005575:cellular_component, GO_0008150:biological_process]
'''

Lors de la création d'une classe ou d'une propriété, l'attribut 'namespace' est utilisé pour construire l'IRI complet de la classe et pour définir dans quelle ontologie la classe est définie (les triples RDF sont ajoutés à cette ontologie). L'IRI de la classe est égal à l'IRI de base de l'espace de noms (base_iri) + le nom de la classe.

Nous visons à définir quatre méthodes de classe dans la classe GO_0005575 pour obtenir des relations plus informatives que les relations directes, qu'on a introduit dans la presentation du TP.

On vise a ajouter 4 méthodes:

* ```sous_parties()``` permet de récupérer l’ensemble des sous-parties du composant. Cette méthode prend en compte les relations BFO_0000051 (has part), mais aussi les relations BFO_0000050 (part of) lues à l’envers, contrairement à ce que nous aurions obtenu avec .INDIRECT_BFO_0000051.
* ```sous_parties_transitives()``` retourne les sous-parties, en prenant en compte les classes filles et la transitivité (si A est une sous-partie de B et que B est une sous-partie de C, alors A est aussi une sous-partie de C).
* ```super_parties()``` et ```super_parties_transitives()``` fonctionnent de la même manière pour les superparties.

<span style="color:red">Nous présentons 3 de ces 4 méthodes, vous devrez compléter la méthode restante en vous inspirant des méthodes que nous présentons et en utilisant la documentation OWLready2.</span>

In [ ]:
with obo:
    class GO_0005575(Thing):
        @classmethod
        def sous_parties(self):
            resultats = list(self.BFO_0000051)
            resultats.extend(self.inverse_restrictions(obo.BFO_0000050))
            return resultats
        
        @classmethod   
        def sous_parties_transitives(self):
            resultats = set()
            for descendant in self.descendants():
                resultats.add(descendant)
                for sous_partie in descendant.sous_parties():
                    resultats.update(sous_partie.sous_parties_transitives())
            return resultats
        
        @classmethod
        def super_parties(self):
            # ICI VOTRE CODE
            return resultats
        
        @classmethod
        def super_parties_transitives(self):
            resultats = set()
            for ancetre in self.ancestors():
                if not issubclass(ancetre, GO_0005575): continue 
                resultats.add(ancetre)
                for super_partie in ancetre.super_parties():
                    if issubclass(ancetre, GO_0005575):
                        resultats.update(super_partie.super_parties_transitives())
            return resultats

Essayons de tester ces nouvelles fonctionnalités en examinant les deux types de relations que nous avons créés, les relations transitives et les non transitives.

In [ ]:
nucleole = go.search(label = "nucleolus")[0]
nucleole.sous_parties()

In [ ]:
nucleole.sous_parties_transitives()

Une fois que nous avons ajouté ces méthodes, qui nous permettent d'analyser les sous-parties et les parties transitives de chaque élément, nous pouvons commencer à réaliser le site web.

## 2. Creation du site Web

La ligne ci-dessous est utilisée pour initialiser un objet qui permet d'interagir avec la base de données de gènes en utilisant l'API de [MyGene.info](https://mygene.info/). Ici, une instance de la classe MyGeneInfo est créée et assignée à la variable ```mg```. Cette instance sera utilisée pour effectuer des requêtes à la base de données de gènes via l'API MyGene.info.

En résumé, cette ligne crée une connexion à la base de données de gènes MyGene.info, permettant au script d'interroger et de récupérer récupérer les concepts GO associés à chacune des gènes et des protéines qu’ils codent.

In [ ]:
mg = mygene.MyGeneInfo()

La fonction ```chercher_proteine(nom_proteine)``` prend en argument le nom d'une protéine et utilise l'objet ```mg``` (qui est une instance de ```mygene.MyGeneInfo()```) pour interroger la base de données de gènes via l'API MyGene.info. Elle récupère ensuite les termes GO associés à la composante cellulaire (CC) de la protéine.

In [ ]:
def chercher_proteine(nom_proteine):
    '''
    Cette ligne effectue une requête à la base de données de gènes en utilisant le nom de la protéine (nom_proteine). 
    La requête est spécifiée pour retourner uniquement les informations sur la composante cellulaire (CC) des annotations GO (fields="go.CC.id"), 
    pour l'espèce humaine (species="human"), et la taille de la réponse est limitée à 1 (size=1).
    '''
    r = mg.query('name:"%s"' % nom_proteine, fields = "go.CC.id",species = "human", size = 1)
    
    '''
    Si la réponse ne contient pas d'informations GO (indiquées par la clé "go" dans la première entrée des résultats), 
    la fonction retourne un ensemble vide. 
    '''
    if not "go" in r["hits"][0]: return set()
  
    '''
    Les informations GO de la composante cellulaire sont extraites de la réponse. 
    Si ces informations ne sont pas sous forme de liste, elles sont converties en liste.
    '''
    cc = r["hits"][0]["go"]["CC"]
    if not isinstance(cc, list): cc = [cc]
  
    '''
    La fonction itère à travers chaque élément de la liste d'annotations GO de la composante cellulaire. 
    Pour chaque élément, elle extrait l'identifiant GO (id_go), le convertit en un terme de l'ontologie GO (terme_go) 
    en utilisant l'objet obo, et ajoute ce terme à l'ensemble sites. Si le terme GO n'est pas trouvé dans l'ontologie, il est ignoré.
    '''
    sites = set()
    for dico in cc:
        id_go = dico["id"]
        terme_go = obo[id_go.replace(":", "_")]
        if terme_go: sites.add(terme_go)
    
    return sites

Les champs "go.CC.id", "go.MF.id" et "go.BP.id" représentent les trois grandes parties de GO (respectivement Cellular Components, Molecular Functions et Biological Process). Pour notre site de rencontre, nous n’utiliserons que les "CC".

Pour réaliser l’intersection sémantique des termes GO décrivant les compartiments cellulaires des deux protéines. Une intersection n’est pas suffisante : l’intersection doit prendre en compte à la fois les relations « est un » d’héritage et les relations « partie-de ». Par exemple, une protéine A présente uniquement dans les membranes et une protéine B présente uniquement dans les mitochondries pourront se rencontrer dans la membrane des mitochondries comme le montre le schéma suivant :

<img src="Ressources/ExempleProts.png" alt="schema" style="height: 220px;"/>

La fonction ```intersection_semantique(sites1, sites2)``` prend deux ensembles de sites GO (représentant les composantes cellulaires de deux protéines) en entrée et retourne l'ensemble des termes GO qui sont communs entre ces deux ensembles, tout en éliminant les sous-parties redondantes.

<span style="color:red">Compléter les parties manquantes du code en utilisant les fonctionnes crées dans la première partie du TP</span>

In [ ]:
def intersection_semantique(sites1, sites2):
    '''
    Ces boucles itèrent sur chaque site dans les ensembles sites1 et sites2 respectivement. 
    Pour chaque site, elle récupère toutes les sous-parties transitives de ce site et 
    les ajoute à l'ensemble correspondant (sous_parties1 ou sous_parties2).
    '''
    sous_parties1 = set()
    for site in sites1:
        # ICI VOTRE CODE
    
    sous_parties2 = set()
    for site in sites2:
        # ICI VOTRE CODE
    
    '''
    Cette ligne calcule l'intersection des ensembles de sous-parties transitives des deux ensembles de sites.
    '''
    sites_communs = sous_parties1 & sous_parties2
    
    '''
    Un dictionnaire cache est créé pour stocker les sous-parties transitives de chaque site commun. 
    Cela est fait pour éviter de recalculer les sous-parties transitives plusieurs fois.
    '''
    cache = # ICI VOTRE CODE
  
    '''
    Cette double boucle vérifie pour chaque paire de sites communs s'il existe une relation d'inclusion
    (une est une sous-partie de l'autre) dans le cache. Si tel est le cas, la sous-partie est considérée
    comme redondante et ignorée. L'ensemble résultant, sites_communs_sans_sous_parties, contient les 
    termes GO communs sans les sous-parties redondantes.
    '''
    sites_communs_sans_sous_parties = set()
    for site in sites_communs:
        for site2 in sites_communs:
            if (not site2 is site) and (site in cache[site2]): break
        else:
            sites_communs_sans_sous_parties.add(site)
  
    return sites_communs_sans_sous_parties

Ce script Flask crée une interface web simple permettant à l'utilisateur de saisir le nom de deux protéines. La route "/resultat" utilise les fonctions ```chercher_proteine()``` et ```intersection_semantique()``` pour récupérer les sites GO associés à chaque protéine et affiche les résultats sur une page web. La page de saisie ("/") génère un formulaire avec deux champs de saisie. Lors de la soumission, l'utilisateur est redirigé vers la page de résultats, où les sites GO associés à chaque protéine et les sites GO communs sont affichés dans une page HTML triée et lisible. L'application Flask est exécutée sur le port 5000 du serveur local. 

In [ ]:
app = Flask(__name__)
@app.route('/')
def page_saisie():
    html  = """
<html><body>
  <form action="/resultat">
    Protéine 1: <input type="text" name="prot1"/><br/><br/>
    Protéine 2: <input type="text" name="prot2"/><br/><br/>
    <input type="submit"/>
  </form>
</body></html>"""
    return html
  
@app.route('/resultat')
def page_resultat():
    prot1 = request.args.get("prot1", "")
    prot2 = request.args.get("prot2", "")
  
    sites1 = chercher_proteine(prot1)
    sites2 = chercher_proteine(prot2)
  
    sites_communs = intersection_semantique(sites1, sites2)
  
    html  = """<html><body>"""
    html += """<h3>Site de la protéine 1 (%s)</h3>""" % prot1
    if sites1:
        html += "<br/>".join(sorted(str(site) for site in sites1))
    else:
        html += "(Aucun)<br/>"
  
    html += """<h3>Site de la protéine 2 (%s)</h3>""" % prot2
    if sites2:
        html += "<br/>".join(sorted(str(site) for site in sites2))
    else:
        html += "(Aucun)<br/>"
  
    html += """<h3>Sites de rencontre possibles</h3>"""
    if sites_communs:
        html += "<br/>".join(sorted(str(site) for site in sites_communs))
    else:
        html += "(Aucun)<br/>"
    
    html += """</body></html>"""
    return html
  
werkzeug.serving.run_simple("localhost", 5000, app)